In [53]:
from delta_lake_spark_utils import *

In [54]:
spark = sparksession_with_delta_lake()


SparkSession created successfully with Delta Lake support!


In [55]:
create_schema = """
create schema if not exists banking
location '../delta_tables/banking'
"""
spark.sql(create_schema)

DataFrame[]

In [56]:
#spark.sql("drop table if exists accounts")
accounts_table = """
CREATE TABLE if not exists banking.accounts (
  account_id INT,
  customer_id INT,
  open_date DATE,
  close_date DATE
) using delta
"""
spark.sql("drop table banking.accounts")
spark.sql(accounts_table)

DataFrame[]

In [57]:
insert_data_account = """
INSERT INTO banking.accounts VALUES
 (100,1,'2020-01-15',NULL),
 (101,2,'2024-03-01',NULL),
 (102,3,'2022-06-10',NULL),
 (103,4,'2023-06-10',NULL)
"""

spark.sql(insert_data_account)
spark.sql(""" select * from banking.accounts """).show()

+----------+-----------+----------+----------+
|account_id|customer_id| open_date|close_date|
+----------+-----------+----------+----------+
|       103|          4|2023-06-10|      NULL|
|       100|          1|2020-01-15|      NULL|
|       101|          2|2024-03-01|      NULL|
|       102|          3|2022-06-10|      NULL|
+----------+-----------+----------+----------+



In [58]:

transaction_table = f"""
CREATE TABLE if not exists banking.txns (
  txn_id INT,
  account_id INT,
  txn_date DATE,
  amount DECIMAL(10,2),
  type String
) using delta
"""
spark.sql("drop table if exists banking.txns")
spark.sql(transaction_table)

DataFrame[]

In [59]:
insert_into_txn = """
INSERT INTO banking.txns VALUES
 (201,100,'2024-10-01',100.00,'deposit'),
 (202,100,'2025-03-15',200.00,'deposit'),
 (203,101,'2025-01-10',300.00,'withdrawal'),
 (204,102,'2023-11-05',400.00,'deposit'),
(205,102,'2023-11-06',1100.00,'deposit')
"""
spark.sql(insert_into_txn)
spark.sql("""select * from banking.txns""").show()

+------+----------+----------+-------+----------+
|txn_id|account_id|  txn_date| amount|      type|
+------+----------+----------+-------+----------+
|   203|       101|2025-01-10| 300.00|withdrawal|
|   201|       100|2024-10-01| 100.00|   deposit|
|   204|       102|2023-11-05| 400.00|   deposit|
|   205|       102|2023-11-06|1100.00|   deposit|
|   202|       100|2025-03-15| 200.00|   deposit|
+------+----------+----------+-------+----------+



> Problem: Find accounts with no transactions in the last 12 months (as of 2025-06-18)

In [60]:
tables = ["accounts", "txns"]
select_tables = [f"select * from banking.{table}" for table in tables]
for select_table in select_tables:
    spark.sql(select_table).show(truncate=False)

+----------+-----------+----------+----------+
|account_id|customer_id|open_date |close_date|
+----------+-----------+----------+----------+
|103       |4          |2023-06-10|NULL      |
|100       |1          |2020-01-15|NULL      |
|101       |2          |2024-03-01|NULL      |
|102       |3          |2022-06-10|NULL      |
+----------+-----------+----------+----------+

+------+----------+----------+-------+----------+
|txn_id|account_id|txn_date  |amount |type      |
+------+----------+----------+-------+----------+
|203   |101       |2025-01-10|300.00 |withdrawal|
|201   |100       |2024-10-01|100.00 |deposit   |
|204   |102       |2023-11-05|400.00 |deposit   |
|205   |102       |2023-11-06|1100.00|deposit   |
|202   |100       |2025-03-15|200.00 |deposit   |
+------+----------+----------+-------+----------+



In [61]:
accounts_no_txn_12_months  =  """
with cte_latest_txn as (
select *
from (
        select account_id,
               txn_id,
                txn_date,
               row_number() over(partition by account_id order by  txn_date desc) as latest_txn
        from banking.txns
    )
where latest_txn = 1
)

select acc.account_id,
       ltxn.txn_date as last_transaction_date,
        current_date,
       current_date - interval 12 months

from banking.accounts as acc
left join cte_latest_txn as ltxn
on acc.account_id = ltxn.account_id
where
    ltxn.txn_date < (current_date - interval 12 months )

"""
spark.sql(accounts_no_txn_12_months).show(truncate=False)

+----------+---------------------+--------------+------------------------------------+
|account_id|last_transaction_date|current_date()|current_date() - INTERVAL '12' MONTH|
+----------+---------------------+--------------+------------------------------------+
|102       |2023-11-06           |2025-06-20    |2024-06-20                          |
+----------+---------------------+--------------+------------------------------------+

